In [282]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from finta import TA
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import jaccard_score
import seaborn as sn
from tabulate import tabulate
from xgboost import XGBClassifier
from ta import add_all_ta_features
from sklearn.feature_selection import RFE
import xgboost as xgb

In [283]:
WINDOW = 8 # number of rows to look ahead to see what the price did
FETCH_INTERVAL = "60m"  # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
INTERVAL = '2y'     # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
symbol = 'AAPL'      # Symbol of the desired stock

# one day 16 rows of data

In [284]:
data = yf.download(  # or pdr.get_data_yahoo(...
        tickers = symbol,


        period = INTERVAL,

        interval = FETCH_INTERVAL,

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        # auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2019-08-20 04:00:00-04:00,52.650000,52.785000,52.650000,52.785000,52.785000,0
2019-08-20 05:00:00-04:00,52.772500,52.772500,52.592500,52.635000,52.635000,0
2019-08-20 06:00:00-04:00,52.627500,52.750000,52.627500,52.687500,52.687500,0
2019-08-20 07:00:00-04:00,52.727500,52.727500,52.500000,52.575000,52.575000,0
2019-08-20 08:00:00-04:00,52.575000,52.850000,52.537500,52.850000,52.850000,0
...,...,...,...,...,...,...
2021-08-19 14:30:00-04:00,146.089996,147.240005,145.839996,147.229996,147.229996,9434215
2021-08-19 15:30:00-04:00,147.223999,147.300003,146.350006,146.669998,146.669998,7259879
2021-08-19 16:00:00-04:00,146.700000,146.880000,146.430000,146.740000,146.740000,3220568


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2019-08-20 04:00:00-04:00,52.650000,52.785000,52.650000,52.785000,52.785000,0
2019-08-20 05:00:00-04:00,52.772500,52.772500,52.592500,52.635000,52.635000,0
2019-08-20 06:00:00-04:00,52.627500,52.750000,52.627500,52.687500,52.687500,0
2019-08-20 07:00:00-04:00,52.727500,52.727500,52.500000,52.575000,52.575000,0
2019-08-20 08:00:00-04:00,52.575000,52.850000,52.537500,52.850000,52.850000,0
...,...,...,...,...,...,...
2021-08-19 14:30:00-04:00,146.089996,147.240005,145.839996,147.229996,147.229996,9434215
2021-08-19 15:30:00-04:00,147.223999,147.300003,146.350006,146.669998,146.669998,7259879
2021-08-19 16:00:00-04:00,146.700000,146.880000,146.430000,146.740000,146.740000,3220568


In [285]:
# data = pd.read_csv('C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\preprocess_new\\AAPL_2y_8_11_05_2021 09_00_35_full.csv', index_col='Datetime')
# data

In [286]:
data.rename(columns={"Close": 'close', "High": 'high', "Low": 'low', 'Volume': 'volume', 'Open': 'open'}, inplace=True)
data.head(10)
important_columns = ['open', 'high', 'low','close','volume']

In [287]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8378 entries, 2019-08-20 04:00:00-04:00 to 2021-08-19 18:00:00-04:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       8378 non-null   float64
 1   high       8378 non-null   float64
 2   low        8378 non-null   float64
 3   close      8378 non-null   float64
 4   Adj Close  8378 non-null   float64
 5   volume     8378 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 458.2 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8378 entries, 2019-08-20 04:00:00-04:00 to 2021-08-19 18:00:00-04:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   open       8378 non-null   float64
 1   high       8378 non-null   float64
 2   low        8378 non-null   float64
 3   close      8378 non-null   float64
 4   Adj Close  8378 non-null   float64
 5   volume     8378 non-null   int64  


In [288]:

def calculate_diffs(diff_number, col_name):
    new_col_name = f'{col_name}_{diff_number}'
    data[new_col_name] = data[col_name].diff(diff_number)

In [289]:
# for name in important_columns:
#     for i in range(1, 11):
#         calculate_diffs(i, name)
#
# data.head(10)

In [290]:
data = data.dropna()

In [291]:
# data
# data.set_index(pd.DatetimeIndex(data['Datetime']))
# data

In [292]:
# fig, ax = plt.subplots()
#
# data.plot(kind='line',x='Datetime',y='close',ax=ax)
#
# every_nth = 4
# for n, label in enumerate(ax.xaxis.get_ticklabels()):
#     if n % every_nth != 0:
#         label.set_visible(False)
#
# plt.xticks(rotation=90)
# plt.show()

In [293]:
# import seaborn as sns
# sns.set()
# ax = data['open'].plot()
# plt.xticks(rotation=90)
# plt.ylabel('Kurs akcji [$]', rotation=90)
# plt.xlabel('Czas')
# plt.legend(['Close'], ncol=1, loc='upper left');
# plt.show()
#
# fig = ax.get_figure()
# fig.savefig('asdf.png', dpi=300, bbox_inches="tight")
#

In [294]:
# data['close_pct'] = data['close'].pct_change()
# data.head()

In [295]:
data.describe()

,open,high,low,close,Adj Close,volume
count,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8.378000e+03
mean,100.865371,101.282155,100.360707,100.869565,100.869565,3.805074e+06
std,29.991512,30.355445,29.821189,29.988044,29.988044,6.932491e+06
min,50.250000,50.467500,50.250000,50.357500,50.357500,0.000000e+00
25%,71.331250,71.678751,71.090000,71.386250,71.386250,0.000000e+00
50%,111.130014,111.683750,110.298750,111.149526,111.149526,0.000000e+00
75%,126.818750,127.180000,126.280000,126.809999,126.809999,5.459065e+06
max,151.410000,438.440000,151.290000,151.670000,151.670000,9.845401e+07


,open,high,low,close,Adj Close,volume
count,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8.378000e+03
mean,100.865377,101.282155,100.360707,100.869565,100.869565,3.805074e+06
std,29.991521,30.355445,29.821189,29.988044,29.988044,6.932491e+06
min,50.250000,50.467500,50.250000,50.357500,50.357500,0.000000e+00
25%,71.331250,71.678751,71.090000,71.386250,71.386250,0.000000e+00
50%,111.130014,111.683750,110.298750,111.149526,111.149526,0.000000e+00
75%,126.818750,127.180000,126.280000,126.809999,126.809999,5.459065e+06
max,151.410000,438.440000,151.290000,151.670000,151.670000,9.845401e+07


In [296]:
from ta import add_all_ta_features

def _get_indicator_data(data):

    data = add_all_ta_features(
        data, open="open", high="high", low="low", close="close", volume="volume", fillna=True)

    return data


In [297]:
data

,open,high,low,close,Adj Close,volume
Datetime,,,,,,
2019-08-20 04:00:00-04:00,52.650000,52.785000,52.650000,52.785000,52.785000,0
2019-08-20 05:00:00-04:00,52.772500,52.772500,52.592500,52.635000,52.635000,0
2019-08-20 06:00:00-04:00,52.627500,52.750000,52.627500,52.687500,52.687500,0
2019-08-20 07:00:00-04:00,52.727500,52.727500,52.500000,52.575000,52.575000,0
2019-08-20 08:00:00-04:00,52.575000,52.850000,52.537500,52.850000,52.850000,0
...,...,...,...,...,...,...
2021-08-19 14:30:00-04:00,146.089996,147.240005,145.839996,147.229996,147.229996,9434215
2021-08-19 15:30:00-04:00,147.223999,147.300003,146.350006,146.669998,146.669998,7259879
2021-08-19 16:00:00-04:00,146.700000,146.880000,146.430000,146.740000,146.740000,3220568


,open,high,low,close,Adj Close,volume
Datetime,,,,,,
2019-08-20 04:00:00-04:00,52.650000,52.785000,52.650000,52.785000,52.785000,0
2019-08-20 05:00:00-04:00,52.772500,52.772500,52.592500,52.635000,52.635000,0
2019-08-20 06:00:00-04:00,52.627500,52.750000,52.627500,52.687500,52.687500,0
2019-08-20 07:00:00-04:00,52.727500,52.727500,52.500000,52.575000,52.575000,0
2019-08-20 08:00:00-04:00,52.575000,52.850000,52.537500,52.850000,52.850000,0
...,...,...,...,...,...,...
2021-08-19 14:30:00-04:00,146.089996,147.240005,145.839996,147.229996,147.229996,9434215
2021-08-19 15:30:00-04:00,147.223999,147.300003,146.350006,146.669998,146.669998,7259879
2021-08-19 16:00:00-04:00,146.700000,146.880000,146.430000,146.740000,146.740000,3220568


In [298]:
data.columns


Index(['open', 'high', 'low', 'close', 'Adj Close', 'volume'], dtype='object')

Index(['open', 'high', 'low', 'close', 'Adj Close', 'volume'], dtype='object')

In [299]:
def create_class_column(row, lowest_threshold, higher_threshold):
    if row['close_shift'] - row['close'] > higher_threshold:
        return 1
    if row['close_shift'] - row['close'] < lowest_threshold:
        return -1
    else:
        return 0

In [300]:
data = _get_indicator_data(data)
data['close_shift'] = data.shift(-WINDOW)['close']
data

C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,open,high,low,close,Adj Close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,close_shift
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-20 04:00:00-04:00,52.650000,52.785000,52.650000,52.785000,52.785000,0,0.000000e+00,0,0.000000,0.000000e+00,...,0.000000,52.785000,0.000000,0.000000,0.000000,0.000000,-47.670043,0.000000,0.000000,53.227524
2019-08-20 05:00:00-04:00,52.772500,52.772500,52.592500,52.635000,52.635000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,0.000000,52.718666,0.000000,0.000000,0.000000,0.000000,-0.284172,-0.284576,-0.284172,53.022251
2019-08-20 06:00:00-04:00,52.627500,52.750000,52.627500,52.687500,52.687500,0,0.000000e+00,0,0.000000,-0.000000e+00,...,0.000000,52.704641,0.000000,0.000000,0.000000,0.000000,0.099744,0.099694,-0.184712,52.853874
2019-08-20 07:00:00-04:00,52.727500,52.727500,52.500000,52.575000,52.575000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,0.000000,52.646839,0.000000,0.000000,0.000000,0.000000,-0.213523,-0.213751,-0.397840,52.876225
2019-08-20 08:00:00-04:00,52.575000,52.850000,52.537500,52.850000,52.850000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,0.000000,52.734980,0.000000,0.000000,0.000000,0.000000,0.523062,0.521699,0.123141,52.587502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-19 14:30:00-04:00,146.089996,147.240005,145.839996,147.229996,147.229996,9434215,1.740003e+09,508862909,0.030041,2.373374e+06,...,-1.097274,145.636372,0.546333,22.097190,7.988910,14.108281,0.780340,0.777312,178.923929,NaN
2021-08-19 15:30:00-04:00,147.223999,147.300003,146.350006,146.669998,146.669998,7259879,1.737634e+09,501603030,0.051486,1.453532e+06,...,-0.862754,145.684068,1.410494,20.047077,10.400543,9.646534,-0.380356,-0.381081,177.863026,NaN
2021-08-19 16:00:00-04:00,146.700000,146.880000,146.430000,146.740000,146.740000,3220568,1.738850e+09,504823598,0.006312,1.278091e+06,...,-0.849048,145.758540,1.011909,13.673612,11.055157,2.618455,0.047727,0.047716,177.995643,NaN


C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,open,high,low,close,Adj Close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,close_shift
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-20 04:00:00-04:00,52.650000,52.785000,52.650000,52.785000,52.785000,0,0.000000e+00,0,0.000000,0.000000e+00,...,0.000000,52.785000,0.000000,0.000000,0.000000,0.000000,-47.670043,0.000000,0.000000,53.227524
2019-08-20 05:00:00-04:00,52.772500,52.772500,52.592500,52.635000,52.635000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,0.000000,52.718666,0.000000,0.000000,0.000000,0.000000,-0.284172,-0.284576,-0.284172,53.022251
2019-08-20 06:00:00-04:00,52.627500,52.750000,52.627500,52.687500,52.687500,0,0.000000e+00,0,0.000000,-0.000000e+00,...,0.000000,52.704641,0.000000,0.000000,0.000000,0.000000,0.099744,0.099694,-0.184712,52.853874
2019-08-20 07:00:00-04:00,52.727500,52.727500,52.500000,52.575000,52.575000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,0.000000,52.646839,0.000000,0.000000,0.000000,0.000000,-0.213523,-0.213751,-0.397840,52.876225
2019-08-20 08:00:00-04:00,52.575000,52.850000,52.537500,52.850000,52.850000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,0.000000,52.734980,0.000000,0.000000,0.000000,0.000000,0.523062,0.521699,0.123141,52.587502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-19 14:30:00-04:00,146.089996,147.240005,145.839996,147.229996,147.229996,9434215,1.740003e+09,508862909,0.030041,2.373374e+06,...,-1.097274,145.636372,0.546333,22.097190,7.988910,14.108281,0.780340,0.777312,178.923929,NaN
2021-08-19 15:30:00-04:00,147.223999,147.300003,146.350006,146.669998,146.669998,7259879,1.737634e+09,501603030,0.051486,1.453532e+06,...,-0.862754,145.684068,1.410494,20.047077,10.400543,9.646534,-0.380356,-0.381081,177.863026,NaN
2021-08-19 16:00:00-04:00,146.700000,146.880000,146.430000,146.740000,146.740000,3220568,1.738850e+09,504823598,0.006312,1.278091e+06,...,-0.849048,145.758540,1.011909,13.673612,11.055157,2.618455,0.047727,0.047716,177.995643,NaN


In [301]:
higher_threshold = 1.5
lowest_threshold = -1.5
last_values_higher = []
last_values_lower = []
data['class_column'] = data.apply((lambda x: create_class_column(x, lowest_threshold, higher_threshold)), axis=1)
while True:
    # print(data['class_column'].value_counts())
    class_counts = data['class_column'].value_counts()
    if abs(class_counts[0] - class_counts[1]) < 15 and abs(class_counts[0] - class_counts[-1]) < 15:
        break

    if len(last_values_higher) == 3:
        last_values_higher.pop(0)
    if len(last_values_lower) == 3:
        last_values_lower.pop(0)

    last_values_higher.append(higher_threshold)
    last_values_lower.append(lowest_threshold)
    if class_counts[0] > class_counts[1]:
        higher_threshold -= 0.01
    if class_counts[0] > class_counts[-1]:
        lowest_threshold += 0.01
    if class_counts[0] < class_counts[1]:
        higher_threshold += 0.01
    if class_counts[0] < class_counts[-1]:
        lowest_threshold -= 0.01

    if higher_threshold in last_values_higher and lowest_threshold in last_values_lower:
        break
    # print(higher_threshold, lowest_threshold)
    data['class_column'] = data.apply((lambda x: create_class_column(x, lowest_threshold, higher_threshold)), axis=1)

data

,open,high,low,close,Adj Close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,close_shift,class_column
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-20 04:00:00-04:00,52.650000,52.785000,52.650000,52.785000,52.785000,0,0.000000e+00,0,0.000000,0.000000e+00,...,52.785000,0.000000,0.000000,0.000000,0.000000,-47.670045,0.000000,0.000000,53.227524,0
2019-08-20 05:00:00-04:00,52.772500,52.772500,52.592500,52.635000,52.635000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.718704,0.000000,0.000000,0.000000,0.000000,-0.284172,-0.284576,-0.284172,53.022251,0
2019-08-20 06:00:00-04:00,52.627500,52.750000,52.627500,52.687500,52.687500,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.704670,0.000000,0.000000,0.000000,0.000000,0.099744,0.099694,-0.184712,52.853874,0
2019-08-20 07:00:00-04:00,52.727500,52.727500,52.500000,52.575000,52.575000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.646889,0.000000,0.000000,0.000000,0.000000,-0.213523,-0.213751,-0.397840,52.876225,0
2019-08-20 08:00:00-04:00,52.575000,52.850000,52.537500,52.850000,52.850000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.734957,0.000000,0.000000,0.000000,0.000000,0.523062,0.521699,0.123141,52.587502,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-19 14:30:00-04:00,146.089996,147.240005,145.839996,147.229996,147.229996,9434215,1.740003e+09,508862909,0.030041,2.373374e+06,...,145.636372,0.546333,22.097190,7.988910,14.108281,0.780340,0.777312,178.923929,NaN,0
2021-08-19 15:30:00-04:00,147.223999,147.300003,146.350006,146.669998,146.669998,7259879,1.737634e+09,501603030,0.051486,1.453532e+06,...,145.684068,1.410494,20.047077,10.400543,9.646534,-0.380356,-0.381081,177.863026,NaN,0
2021-08-19 16:00:00-04:00,146.700000,146.880000,146.430000,146.740000,146.740000,3220568,1.738850e+09,504823598,0.006312,1.278091e+06,...,145.758540,1.011909,13.673612,11.055157,2.618455,0.047727,0.047716,177.995643,NaN,0


,open,high,low,close,Adj Close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,close_shift,class_column
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-20 04:00:00-04:00,52.650000,52.785000,52.650000,52.785000,52.785000,0,0.000000e+00,0,0.000000,0.000000e+00,...,52.785000,0.000000,0.000000,0.000000,0.000000,-47.670043,0.000000,0.000000,53.227524,0
2019-08-20 05:00:00-04:00,52.772500,52.772500,52.592500,52.635000,52.635000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.718666,0.000000,0.000000,0.000000,0.000000,-0.284172,-0.284576,-0.284172,53.022251,0
2019-08-20 06:00:00-04:00,52.627500,52.750000,52.627500,52.687500,52.687500,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.704641,0.000000,0.000000,0.000000,0.000000,0.099744,0.099694,-0.184712,52.853874,0
2019-08-20 07:00:00-04:00,52.727500,52.727500,52.500000,52.575000,52.575000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.646839,0.000000,0.000000,0.000000,0.000000,-0.213523,-0.213751,-0.397840,52.876225,0
2019-08-20 08:00:00-04:00,52.575000,52.850000,52.537500,52.850000,52.850000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.734980,0.000000,0.000000,0.000000,0.000000,0.523062,0.521699,0.123141,52.587502,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-19 14:30:00-04:00,146.089996,147.240005,145.839996,147.229996,147.229996,9434215,1.740003e+09,508862909,0.030041,2.373374e+06,...,145.636372,0.546333,22.097190,7.988910,14.108281,0.780340,0.777312,178.923929,NaN,0
2021-08-19 15:30:00-04:00,147.223999,147.300003,146.350006,146.669998,146.669998,7259879,1.737634e+09,501603030,0.051486,1.453532e+06,...,145.684068,1.410494,20.047077,10.400543,9.646534,-0.380356,-0.381081,177.863026,NaN,0
2021-08-19 16:00:00-04:00,146.700000,146.880000,146.430000,146.740000,146.740000,3220568,1.738850e+09,504823598,0.006312,1.278091e+06,...,145.758540,1.011909,13.673612,11.055157,2.618455,0.047727,0.047716,177.995643,NaN,0


,open,high,low,close,Adj Close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,close_shift,class_column
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-08-20 04:00:00-04:00,52.650000,52.785000,52.650000,52.785000,52.785000,0,0.000000e+00,0,0.000000,0.000000e+00,...,52.785000,0.000000,0.000000,0.000000,0.000000,-47.670043,0.000000,0.000000,53.227524,0
2019-08-20 05:00:00-04:00,52.772500,52.772500,52.592500,52.635000,52.635000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.718666,0.000000,0.000000,0.000000,0.000000,-0.284172,-0.284576,-0.284172,53.022251,0
2019-08-20 06:00:00-04:00,52.627500,52.750000,52.627500,52.687500,52.687500,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.704641,0.000000,0.000000,0.000000,0.000000,0.099744,0.099694,-0.184712,52.853874,0
2019-08-20 07:00:00-04:00,52.727500,52.727500,52.500000,52.575000,52.575000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.646839,0.000000,0.000000,0.000000,0.000000,-0.213523,-0.213751,-0.397840,52.876225,0
2019-08-20 08:00:00-04:00,52.575000,52.850000,52.537500,52.850000,52.850000,0,0.000000e+00,0,0.000000,-0.000000e+00,...,52.734980,0.000000,0.000000,0.000000,0.000000,0.523062,0.521699,0.123141,52.587502,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-19 14:30:00-04:00,146.089996,147.240005,145.839996,147.229996,147.229996,9434215,1.740003e+09,508862909,0.030041,2.373374e+06,...,145.636372,0.546333,22.097190,7.988910,14.108281,0.780340,0.777312,178.923929,NaN,0
2021-08-19 15:30:00-04:00,147.223999,147.300003,146.350006,146.669998,146.669998,7259879,1.737634e+09,501603030,0.051486,1.453532e+06,...,145.684068,1.410494,20.047077,10.400543,9.646534,-0.380356,-0.381081,177.863026,NaN,0
2021-08-19 16:00:00-04:00,146.700000,146.880000,146.430000,146.740000,146.740000,3220568,1.738850e+09,504823598,0.006312,1.278091e+06,...,145.758540,1.011909,13.673612,11.055157,2.618455,0.047727,0.047716,177.995643,NaN,0


In [302]:
# Class divide
data['class_column'] = data.apply((lambda x: create_class_column(x, lowest_threshold, higher_threshold)), axis=1)

data['class_column'].value_counts()

 0    2823
 1    2798
-1    2757
Name: class_column, dtype: int64

 0    2823
 1    2798
-1    2757
Name: class_column, dtype: int64

 0    2823
 1    2798
-1    2757
Name: class_column, dtype: int64

In [303]:
filename_to_export = f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\{symbol}_{INTERVAL}_{WINDOW}_diff_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}_full.csv'
data.to_csv(filename_to_export, index=True)
print(filename_to_export)

C:\Users\exomat\Desktop\repo\magisterka_analiza\data\results\train_test\AAPL_2y_8_diff_20_08_2021 00_06_50_full.csv
C:\Users\exomat\Desktop\repo\magisterka_analiza\data\results\train_test\AAPL_2y_8_diff_20_08_2021 00_14_25_full.csv
C:\Users\exomat\Desktop\repo\magisterka_analiza\data\results\train_test\AAPL_2y_8_diff_20_08_2021 00_25_27_full.csv


In [304]:

# Class divide
data['class_column'].value_counts()

 0    2823
 1    2798
-1    2757
Name: class_column, dtype: int64

 0    2823
 1    2798
-1    2757
Name: class_column, dtype: int64

 0    2823
 1    2798
-1    2757
Name: class_column, dtype: int64

In [305]:
# del (data['close'])
del (data['close_shift'])
data = data.dropna()

In [306]:
data['class_column'].value_counts()


 0    2823
 1    2798
-1    2757
Name: class_column, dtype: int64

 0    2823
 1    2798
-1    2757
Name: class_column, dtype: int64

 0    2823
 1    2798
-1    2757
Name: class_column, dtype: int64

In [307]:
y = data['class_column']
features = [x for x in data.columns if x not in ['class_column']]
x = data[features]
# scaler = MinMaxScaler()
# x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)
ROWS_TO_PREDICT = 128
if ROWS_TO_PREDICT == 33:
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.3, shuffle=False)
else:
    x_train= x.iloc[:-ROWS_TO_PREDICT]
    y_train= y.iloc[:-ROWS_TO_PREDICT]
    x_test =x.iloc[-ROWS_TO_PREDICT:]
    y_test=y.iloc[-ROWS_TO_PREDICT:]

In [308]:
classifiers = dict()
# classifiers['DecisionTreeClassifier 1'] = DecisionTreeClassifier(max_depth=15, random_state=0, criterion='gini',
#                                                                  splitter='best')
# classifiers['DecisionTreeClassifier 2'] = DecisionTreeClassifier(max_depth=40, random_state=0, criterion='gini',
#                                                                  splitter='best')
# classifiers['DecisionTreeClassifier 3'] = DecisionTreeClassifier(max_depth=40, random_state=0, criterion='gini',
#                                                                  splitter='random')
# classifiers['DecisionTreeClassifier 4'] = DecisionTreeClassifier(max_depth=15, random_state=0, criterion='entropy',
#                                                                  splitter='best')
# classifiers['DecisionTreeClassifier 5'] = DecisionTreeClassifier(max_depth=40, random_state=0, criterion='entropy',
#                                                                  splitter='best')
#
# classifiers['DecisionTreeClassifier 6'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='gini',splitter='best')
# classifiers['DecisionTreeClassifier 7'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='gini',splitter='random')
# classifiers['DecisionTreeClassifier 8'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='entropy',splitter='best')

#
# classifiers['RandomForestClassifier 1'] = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0,
#                                                                  criterion='gini', n_jobs=-1)
# classifiers['RandomForestClassifier 2'] = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=0,
#                                                                  criterion='gini', n_jobs=-1)
# classifiers['RandomForestClassifier 3'] = RandomForestClassifier(n_estimators=10000, max_depth=3, random_state=0,
#                                                                  criterion='gini', n_jobs=-1)
# classifiers['RandomForestClassifier 4'] = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=0,
#                                                                  criterion='entropy', n_jobs=-1)
# classifiers['RandomForestClassifier 5'] = RandomForestClassifier(n_estimators=10000, max_depth=3, random_state=0,
#                                                                  criterion='entropy', n_jobs=-1)
# classifiers['RandomForestClassifier 6'] = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0,
#                                                                  criterion='entropy', n_jobs=-1)

#
classifiers['GradientBoostingClassifier 1'] = GradientBoostingClassifier(n_estimators=100, random_state=0,
                                                                         criterion='friedman_mse', max_depth=3,
                                                                         learning_rate=0.1)
classifiers['GradientBoostingClassifier 2'] = GradientBoostingClassifier(n_estimators=100, random_state=0,
                                                                         criterion='friedman_mse', max_depth=10,
                                                                         learning_rate=0.3,
                                                                         loss='deviance')
classifiers['GradientBoostingClassifier 3'] = GradientBoostingClassifier(n_estimators=1000, random_state=0,
                                                                         criterion='friedman_mse', max_depth=3,
                                                                         learning_rate=0.5)


# classifiers['XGBRFClassifier 1'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=2, n_estimators=1000, eta=0.2)
# classifiers['XGBRFClassifier 2'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=3, n_estimators=1000, eta=0.2)
# classifiers['XGBRFClassifier 3'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=15, n_estimators=1000, eta=0.3)
# classifiers['XGBRFClassifier 4'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=3, n_estimators=1000, eta=0.2)
# classifiers['XGBRFClassifier 5'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=6, n_estimators=1000, eta=0.3)
# classifiers['XGBRFClassifier 6'] = xgb.sklearn.XGBRFClassifier(n_jobs=-1, max_depth=3, n_estimators=1000, eta=0.6)

# classifiers['XGBClassifier 1'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.2)
# classifiers['XGBClassifier 2'] = xgb.XGBClassifier(nthread=-1, max_depth=10, n_estimators=1000, eta=0.3)
# classifiers['XGBClassifier 3'] = xgb.XGBClassifier(nthread=-1, max_depth=25, n_estimators=1000, eta=0.2)
# classifiers['XGBClassifier 4'] = xgb.XGBClassifier(nthread=-1, max_depth=10, n_estimators=1000, eta=0.5)
# classifiers['XGBClassifier 5'] = xgb.XGBClassifier(nthread=-1, max_depth=6, n_estimators=1000, eta=0.3)
# classifiers['XGBClassifier 6'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.3)

classifiers_boosted = dict()
# classifiers_boosted['GradientBoostingClassifier 1S'] = GradientBoostingClassifier(n_estimators=100, random_state=0,
#                                                                                   criterion='friedman_mse', max_depth=3,
#                                                                                   learning_rate=0.1)
# classifiers_boosted['GradientBoostingClassifier 2S'] = GradientBoostingClassifier(n_estimators=1000, random_state=0,
#                                                                                   criterion='friedman_mse', max_depth=3,
#                                                                                   learning_rate=0.3)
# classifiers_boosted['GradientBoostingClassifier 3S'] = GradientBoostingClassifier(n_estimators=1000, random_state=0,
#                                                                                   criterion='friedman_mse', max_depth=2,
#                                                                                   learning_rate=0.5)
# classifiers_boosted['XGBClassifier 1'] = xgb.XGBClassifier(nthread=-1, max_depth=3, n_estimators=1000, eta=0.2)
# classifiers_boosted['XGBClassifier 2'] = xgb.XGBClassifier(nthread=-1, max_depth=10, n_estimators=1000, eta=0.3)
# classifiers_boosted['XGBClassifier 3'] = xgb.XGBClassifier(nthread=-1, max_depth=25, n_estimators=1000, eta=0.2)

In [309]:
def print_conf_matrix(test_y, predict, name):
    matrix = confusion_matrix(test_y, predict,  labels=[-1, 0, 1])
    print(matrix)
    ax= plt.subplot()
    sn.heatmap(matrix, annot=True, ax = ax) #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels', color='white')
    ax.set_ylabel('True labels', color='white')
    ax.set_title(f'Confusion Matrix for {name}' , color='white')
    ax.xaxis.set_ticklabels(['-1','0', '1'], color='white')
    ax.yaxis.set_ticklabels(['-1','0', '1'], color='white')
    plt.show()

def train_model(model,train_x, train_y):
    model.fit(train_x, train_y)

In [274]:
predictions_train= dict()
score_train = dict()
predictions= dict()
score = dict()
for k,v in classifiers.items():
    print("Calculate: ", k)
    train_model(v,x_train,y_train)
    predictions_train[k] = v.predict(x_train)
    score_train[k] = accuracy_score(y_train.values, predictions_train[k])
    predictions[k] = v.predict(x_test)
    score[k] = accuracy_score(y_test.values, predictions[k])
    print('Score_train: ',  score_train[k] )
    print('Score: ',  score[k] )
    # print_conf_matrix(test_y, predictions[k], k)

Calculate:  GradientBoostingClassifier 1
Score_train:  0.7372100954979536
Score:  0.37828162291169454
Calculate:  GradientBoostingClassifier 2
Score_train:  1.0
Score:  0.35759745425616546
Calculate:  GradientBoostingClassifier 3
Score_train:  1.0
Score:  0.3536197295147176
Calculate:  GradientBoostingClassifier 1
Score_train:  0.6983401491460187
Score:  0.453125
Calculate:  GradientBoostingClassifier 2
Score_train:  1.0
Score:  0.28125
Calculate:  GradientBoostingClassifier 3
Score_train:  1.0
Score:  0.125
Calculate:  GradientBoostingClassifier 1
Score_train:  0.6981818181818182
Score:  0.4609375
Calculate:  GradientBoostingClassifier 2
Score_train:  1.0
Score:  0.4140625
Calculate:  GradientBoostingClassifier 3


In [275]:
filename_to_export = f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\result_test_xrand_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'
filename_to_export_train= f'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\train_test\\result_train_xrand_{symbol}_{WINDOW}_{ROWS_TO_PREDICT}_{datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'

headers = ["Classifier type", "Accuracy"]
score_df = pd.DataFrame(score.items(), columns=headers)
print(tabulate(score_df, headers, tablefmt="psql"))
score_df.to_csv(filename_to_export,mode='a', index=False, header=False)
score_df_train = pd.DataFrame(score_train.items(), columns=headers)
print(tabulate(score_df_train, headers, tablefmt="psql"))
score_df_train.to_csv(filename_to_export_train,mode='a', index=False, header=False)

+----+------------------------------+------------+
|    | Classifier type              |   Accuracy |
|----+------------------------------+------------|
|  0 | GradientBoostingClassifier 1 |   0.378282 |
|  1 | GradientBoostingClassifier 2 |   0.357597 |
|  2 | GradientBoostingClassifier 3 |   0.35362  |
+----+------------------------------+------------+
+----+------------------------------+------------+
|    | Classifier type              |   Accuracy |
|----+------------------------------+------------|
|  0 | GradientBoostingClassifier 1 |    0.73721 |
|  1 | GradientBoostingClassifier 2 |    1       |
|  2 | GradientBoostingClassifier 3 |    1       |
+----+------------------------------+------------+
+----+------------------------------+------------+
|    | Classifier type              |   Accuracy |
|----+------------------------------+------------|
|  0 | GradientBoostingClassifier 1 |   0.453125 |
|  1 | GradientBoostingClassifier 2 |   0.28125  |
|  2 | GradientBoostingClassifi

In [276]:
# rfe = RFE(classifiers['RandomForestClassifier 5'],10)
# fited = rfe.fit(x_train, y_train)
# rfe

In [277]:
# names = x.columns
# columns=[]
# for i in range(len(fited.support_)):
#     if fited.support_[i]:
#         columns.append(names[i])
#
# print("Columns with predictive power:", columns )
# columns = columns + ['high', 'low', 'volume', 'open']
# columns

In [278]:
# x_test_cropped = x_test[columns]
# x_train_cropped = x_train[columns]
# x_train_cropped
#

In [279]:
# for k,v in classifiers_boosted.items():
#     print("Calculate: ", k)
#     train_model(v,x_train_cropped,y_train)
#     predictions_train[k] = v.predict(x_train_cropped)
#     score_train[k] = accuracy_score(y_train.values, predictions_train[k])
#     predictions[k] = v.predict(x_test_cropped)
#     score[k] = accuracy_score(y_test.values, predictions[k])
#     print('Score train: ',  score_train[k] )
#     print('Score: ',  score[k] )
#     # print_conf_matrix(test_y, predictions[k], k)


In [280]:
#
# model = xgb.XGBClassifier(nthread =-1,max_depth=14,n_estimators=1000,
#                           eta =0.2)
# model.fit(x_train,y_train)
# predicted_train = model.predict(x_train)
# predicted_test = model.predict(x_test)
# print("------------")
# print(f'max_depth: {14}')
# score_train['XGBClassifier'] = accuracy_score(y_train.values, predicted_train)
# score['XGBClassifier'] = accuracy_score(y_test.values, predicted_test)
# print(score_train['XGBClassifier'])
# print(score['XGBClassifier'])
# print("------------")
#

In [281]:
# model = xgb.sklearn.XGBRFClassifier(n_jobs=-1,max_depth=12,n_estimators =100,eta=0.4)
# model.fit(x_train,y_train)
# predicted_train = model.predict(x_train)
# predicted_test = model.predict(x_test)
# print("------------")
# print(f'eta: ')
# score_train['XGBRFClassifier'] = accuracy_score(y_train.values, predicted_train)
# score['XGBRFClassifier'] = accuracy_score(y_test.values, predicted_test)
# print(accuracy_score(y_train.values, predicted_train))
# print(accuracy_score(y_test.values, predicted_test))
# print("------------")